# Schnorr digital signatures and verification
## Key pair generation

- $E$ is an elliptic curve, $N$ is the order of the curve
- $G$ is a generator and $h$ is an hash function
- The user chooses his secret key $d$, $0 < d < N$
- The user computes the point $P = dG = (P_x, P_y)$: his public key is the $x$-coordinate, $P_x$

In [2]:
def hashPDF(file, BLOCK_SIZE):
    # hash=sha256()
    with open(file, 'rb') as f: # Open the file to read it's bytes
        fb = f.read(BLOCK_SIZE) # Read from the file. Take in the amount declared above
        M = sl.sha256(fb)
    return M
file = "C:/Users/guzma/OneDrive/Documents/TEC/S6/MA2006/reto/test.pdf"
size = os.path.getsize(file) 

In [28]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))

ckp.create_keypair(n_keys)["users"]

[{'privateKey': 'cb203c0a070b40dbbb57be96f96305ec177d2c6613769762307f80948e1b7a74',
  'publicKey': '019fbce66971c02530b7f1f84380c59ce375a57b6eaba9179be303cab6e95b9e'},
 {'privateKey': '422a908896e3e1a14ae5aca1f0f44e3a9c5f2461ad1207512777caf73f1b8911',
  'publicKey': 'b490aa6768c98cc1cf891c716bb3573e7c971b987584a8a4958294d81c721b40'}]

## Schnorr Sign

- $M$ message, $d$ is the private key, the $x$-coordinate of $P = dG$ is the public key
- $A$, the signer, picks a random $k$, $1 < k < n$ and calculates $R = kG$
- A computes $e = h(R || P || M)$
- A computes $s = (k + ed) \mod n$
- The signature is the pair $(R_x, s)$ 

In [3]:
import create_keypair as ckp
import schnorr_lib as sl

user = ckp.create_keypair(1)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig = sl.schnorr_sign(M, user[0]["privateKey"])

print("PublicKey =", user[0]["publicKey"])
print("Signature =",sig.hex())

PublicKey = <class 'str'>
Signature = 8b40d9d6c49df70d69e6588c189c3fdbbc78ae4d99069f5eec9fc850f3f357ffc4c5277d8ff768616e415257986fe5ab2e14995a38061a4bbce03908e16f3967


## Schnorr MuSig-1

- $L = h(P_1 || ... ||P_n)$, where n is the number of users and $P_i$ is the public key of the $i^{th}$ user
- Every user $i$ computes the quantity $a_i = h(L||P_i)$
- The aggregate $\tilde{X} = \sum\limits_{i=1}^{n} a_i P_i$ is a public parameter
- Every user chooses $r_i$ and computes $R_i = r_i G$
- Every user computes the point $R = R_1+...+R_n =(R_x,R_y)$
- Every user computes $c = h(R || \tilde{X}|| M)$
- Every user computes $s_i = r_i + cd_i a_i \mod N$, then the aggregate is $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$

In [30]:
import create_keypair as ckp
import schnorr_lib as sl

n_keys = int(input("Insert the number of keys to be generated:"))
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X = sl.schnorr_musig_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = c0a1249e920d8b2f51ade69acd3d862f4755bd20cc08f474274fad8cf55fdeea
Signature = d12daa5527ac9ee1735c72e419a5ffa6182c92ea2c8efe8ebd75c5df504093c4a1d21eb7a2d96e16bb043a2bb24e397f91fa5358169ce3e890f44e1907a67de1


## Schnorr MuSig-2

- Compute $L$, $a_i = h(L||P_i )$ and $\tilde{X}$ as before
- Each user $i$ chooses $\nu$ different nonces $r_{i,1}, . . . , r_{i,ν}$ and computes the points
$R_{i,j} = r_{i,j}G$, $\forall j \in \{1,...,\nu\}$
- Compute $R_j = \sum\limits_{i=1}^{n} R_{i,j}, \; \forall j \in \{1,...,\nu\}$
- Compute the hash $b = h(R_1||...||R_{\nu}|| \tilde{X} || M)$, then compute $R = \sum\limits_{j=1}^{\nu} b^{j-1} R_{j}$
- Compute the hash $c = h(R || \tilde{X}||M)$, then every user $i$ computes
$s_i=c a_id_i+\sum\limits_{j=1}^{\nu} r_{i,j}\; b^{j−1} \mod N$
- Compute $s=s_1+...+s_n \mod N$
- The signature is the couple $(R_x,s)$.

In [31]:
import create_keypair as ckp
import schnorr_lib as sl
import os
# from hashlib import sha256

n_keys = int(input("Insert the number of keys to be generated:"))
# n_keys = 2
users = ckp.create_keypair(n_keys)["users"]

# M = input("Insert the message to sign:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

sig, X = sl.schnorr_musig2_sign(M, users)

print("Aggregated key =",X.hex())
print("Signature =",sig.hex())

Aggregated key = 3dd9f89da17633fa5be72986a605afdc3477a2663bd0661198d59a93d0fb3f62
Signature = ca50387b9e098aac7f7396551091f834c0a0c760e40b5fcedb598a3de2fc4136e871d779bfc01300e8806f26779628070fccf65b23fd58595f7deb11f89d6d00


## Verify Schnorr

- $B$, the receiver, computes the point $sG$
- $B$ computes $e = h(R || P || M)$ $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$
- if $sG = R + eP$, the signature is valid  $(P$ is equal to $\tilde{X}$ in the MuSig cases$)$

In [25]:
import schnorr_lib as sl

# M = input("Insert the message to verify:")
# M = sl.sha256(M.encode())
M = hashPDF(file, size)

pubkey = input("Insert the public key (or the aggregated key if MuSig was used):")
pubkey_bytes = bytes.fromhex(pubkey)

sig = input("Insert the generated sign:")
sig_bytes = bytes.fromhex(sig)

result = sl.schnorr_verify(M, pubkey_bytes, sig_bytes)

if result:
    print("The signature is VALID for this message and this public key")
else:
    print("The signature is NOT VALID for this message and this public key")

The signature is VALID for this message and this public key
